## Installation & Setup

Installing required packages for comprehensive analysis:
- **obonet**: Gene Ontology (GO) graph parsing
- **biopython**: Protein sequence handling
- **plotly**: Interactive visualizations

In [ ]:
import sys
!{sys.executable} -m pip install obonet biopython -q
!pip install --upgrade plotly

## Import Required Libraries

Setting up our analysis toolkit with essential libraries for:
- Data manipulation (pandas, numpy)
- Visualization (matplotlib, seaborn, plotly)
- Bioinformatics (Bio, obonet, networkx)

In [ ]:
# Core data science
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Bioinformatics
from Bio import SeqIO
import obonet
import networkx as nx

# Utilities
from collections import Counter, defaultdict
from typing import Dict, List, Tuple
import os
from tqdm.notebook import tqdm

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("All libraries loaded successfully!")

## Data Loading

Loading all competition files and performing initial validation checks.

In [ ]:
# Define paths
BASE_PATH = '/kaggle/input/cafa-6-protein-function-prediction'
TRAIN_PATH = f'{BASE_PATH}/Train'
TEST_PATH = f'{BASE_PATH}/Test'

paths = {
    'train_seq': f'{TRAIN_PATH}/train_sequences.fasta',
    'train_terms': f'{TRAIN_PATH}/train_terms.tsv',
    'train_tax': f'{TRAIN_PATH}/train_taxonomy.tsv',
    'go_obo': f'{TRAIN_PATH}/go-basic.obo',
    'test_seq': f'{TEST_PATH}/testsuperset.fasta',
    'ia': f'{BASE_PATH}/IA.tsv'
}

# Load tabular data
print("Loading datasets...")
train_terms = pd.read_csv(paths['train_terms'], sep='\t')
train_tax = pd.read_csv(paths['train_tax'], sep='\t', header=None, names=['EntryID', 'TaxonID'])
ia_df = pd.read_csv(paths['ia'], sep='\t', header=None, names=['term', 'IA'])

# Load GO graph
print("Loading Gene Ontology graph...")
go_graph = obonet.read_obo(paths['go_obo'])

# Display data overview
print("\n" + "="*70)
print("DATA OVERVIEW")
print("="*70)
print(f"Training Terms:        {len(train_terms):,} annotations")
print(f"Unique Proteins:       {train_terms['EntryID'].nunique():,}")
print(f"Unique GO Terms:       {train_terms['term'].nunique():,}")
print(f"Unique Species (Taxa): {train_tax['TaxonID'].nunique():,}")
print(f"GO Graph Nodes:        {go_graph.number_of_nodes():,}")
print(f"GO Graph Edges:        {go_graph.number_of_edges():,}")
print(f"IA Records:            {len(ia_df):,}")
print("="*70)

# Preview data
display(train_terms.head())

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 25px; border-radius: 12px; margin: 25px 0; box-shadow: 0 6px 15px rgba(0,0,0,0.2);">
  <h1 style="color: white; margin: 0; font-size: 32px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
    Protein Sequence Analysis
  </h1>
  <p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">
    Understanding the building blocks of protein function
  </p>
</div>

### Part 1: Loading and Basic Statistics

Extracting key features from FASTA sequences:
- Sequence lengths
- Amino acid composition
- Dataset statistics

In [ ]:
def load_fasta_stats(fasta_path, max_seqs=None):
    """Load FASTA file and extract sequence statistics"""
    data = []
    aa_counter = Counter()
    
    for i, record in enumerate(tqdm(SeqIO.parse(fasta_path, 'fasta'), desc="Loading sequences")):
        if max_seqs and i >= max_seqs:
            break
            
        entry_id = record.id.split('|')[1] if '|' in record.id else record.id
        seq = str(record.seq)
        
        data.append({
            'EntryID': entry_id,
            'length': len(seq),
            'seq': seq
        })
        aa_counter.update(seq)
    
    return pd.DataFrame(data), aa_counter

# Load training sequences
print("Loading training sequences...")
train_seqs, train_aa_counts = load_fasta_stats(paths['train_seq'])

# Basic statistics
print(f"\nSequence Statistics:")
print(f"   Total sequences: {len(train_seqs):,}")
print(f"   Mean length: {train_seqs['length'].mean():.1f} amino acids")
print(f"   Median length: {train_seqs['length'].median():.0f} amino acids")
print(f"   Min length: {train_seqs['length'].min()}")
print(f"   Max length: {train_seqs['length'].max():,}")
print(f"   Std dev: {train_seqs['length'].std():.1f}")

### Part 2: Sequence Length Distribution

Understanding the distribution of protein lengths helps with:
- Batch size selection for training
- Model architecture decisions (e.g., max sequence length)
- Identifying potential outliers

In [ ]:
# Create comprehensive length visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribution', 'Box Plot', 'Cumulative Distribution', 'Log Scale'),
    specs=[[{"type": "histogram"}, {"type": "box"}],
           [{"type": "scatter"}, {"type": "histogram"}]]
)

# Histogram
fig.add_trace(
    go.Histogram(x=train_seqs['length'], nbinsx=100, name='Count',
                 marker_color='lightblue', showlegend=False),
    row=1, col=1
)

# Box plot
fig.add_trace(
    go.Box(y=train_seqs['length'], name='Length', marker_color='lightcoral',
           boxmean='sd', showlegend=False),
    row=1, col=2
)

# Cumulative distribution
sorted_lengths = np.sort(train_seqs['length'])
cumulative = np.arange(1, len(sorted_lengths) + 1) / len(sorted_lengths) * 100
fig.add_trace(
    go.Scatter(x=sorted_lengths, y=cumulative, mode='lines',
               name='Cumulative %', line=dict(color='green', width=2),
               showlegend=False),
    row=2, col=1
)

# Log scale histogram
fig.add_trace(
    go.Histogram(x=train_seqs['length'], nbinsx=100, name='Count (log)',
                 marker_color='mediumpurple', showlegend=False),
    row=2, col=2
)

fig.update_xaxes(title_text="Sequence Length", row=2, col=1)
fig.update_xaxes(title_text="Sequence Length (log)", type="log", row=2, col=2)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Cumulative %", row=2, col=1)

fig.update_layout(height=800, title_text="Protein Sequence Length Analysis", showlegend=False)
fig.show()

# Key percentiles
percentiles = train_seqs['length'].quantile([0.25, 0.5, 0.75, 0.90, 0.95, 0.99])
print("\nKey Percentiles:")
for p, val in percentiles.items():
    print(f"   {int(p*100)}th percentile: {val:.0f} amino acids")

### Part 3: Amino Acid Composition Analysis

The 20 standard amino acids and their properties:
- **Hydrophobic**: A, V, I, L, M, F, W, P
- **Polar**: S, T, C, Y, N, Q
- **Charged (+)**: K, R, H
- **Charged (-)**: D, E
- **Special**: G (smallest)

Understanding amino acid distribution can reveal:
- Dataset biases
- Protein type composition
- Potential feature engineering opportunities

In [ ]:
# Calculate amino acid frequencies
aa_total = sum(train_aa_counts.values())
aa_freq = {aa: count/aa_total*100 for aa, count in train_aa_counts.items()}

# Sort by frequency
aa_sorted = sorted(aa_freq.items(), key=lambda x: x[1], reverse=True)
amino_acids = [x[0] for x in aa_sorted]
frequencies = [x[1] for x in aa_sorted]

# Amino acid properties
aa_properties = {
    'L': 'Hydrophobic', 'A': 'Hydrophobic', 'G': 'Special', 'V': 'Hydrophobic',
    'S': 'Polar', 'E': 'Charged(-)', 'I': 'Hydrophobic', 'K': 'Charged(+)',
    'R': 'Charged(+)', 'T': 'Polar', 'D': 'Charged(-)', 'P': 'Hydrophobic',
    'N': 'Polar', 'Q': 'Polar', 'F': 'Hydrophobic', 'Y': 'Polar',
    'M': 'Hydrophobic', 'H': 'Charged(+)', 'C': 'Polar', 'W': 'Hydrophobic'
}

colors = {
    'Hydrophobic': '#3498db',
    'Polar': '#2ecc71',
    'Charged(+)': '#e74c3c',
    'Charged(-)': '#f39c12',
    'Special': '#9b59b6'
}

bar_colors = [colors[aa_properties.get(aa, 'Special')] for aa in amino_acids]

# Create visualization
fig = go.Figure()

fig.add_trace(go.Bar(
    x=amino_acids,
    y=frequencies,
    marker_color=bar_colors,
    text=[f'{f:.2f}%' for f in frequencies],
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Frequency: %{y:.3f}%<br>Property: ' + 
                  '<extra></extra>'
))

fig.update_layout(
    title='Amino Acid Composition in Training Proteins',
    xaxis_title='Amino Acid',
    yaxis_title='Frequency (%)',
    height=500,
    showlegend=False
)

fig.show()

# Group by property
property_freq = defaultdict(float)
for aa, freq in aa_freq.items():
    prop = aa_properties.get(aa, 'Special')
    property_freq[prop] += freq

print("\nAmino Acid Composition by Property:")
for prop, freq in sorted(property_freq.items(), key=lambda x: x[1], reverse=True):
    print(f"   {prop:15s}: {freq:5.2f}%")

<div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); padding: 25px; border-radius: 12px; margin: 25px 0; box-shadow: 0 6px 15px rgba(0,0,0,0.2);">
  <h1 style="color: white; margin: 0; font-size: 32px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
    GO Terms & Label Analysis
  </h1>
  <p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">
    Understanding the prediction targets and class distribution
  </p>
</div>

### Part 1: Class Imbalance & Distribution

The CAFA challenge involves predicting GO terms across three subontologies:
- **MFO**: Molecular Function (What does the protein do?)
- **BPO**: Biological Process (What pathway is it involved in?)
- **CCO**: Cellular Component (Where is it located?)

Understanding class imbalance is crucial for:
- Choosing appropriate loss functions
- Setting up balanced validation sets
- Calibrating prediction thresholds

In [ ]:
# Map aspect codes
aspect_map = {'F': 'MFO', 'P': 'BPO', 'C': 'CCO'}
train_terms['aspect_name'] = train_terms['aspect'].map(aspect_map)

# Overall statistics
print("="*70)
print("LABEL DISTRIBUTION STATISTICS")
print("="*70)
print(f"Total annotations:        {len(train_terms):,}")
print(f"Unique proteins:          {train_terms['EntryID'].nunique():,}")
print(f"Unique GO terms:          {train_terms['term'].nunique():,}")
print(f"Avg terms per protein:    {len(train_terms) / train_terms['EntryID'].nunique():.2f}")
print("="*70)

# Aspect distribution
aspect_counts = train_terms['aspect_name'].value_counts()
print("\nDistribution by Subontology:")
for aspect, count in aspect_counts.items():
    pct = count / len(train_terms) * 100
    print(f"   {aspect}: {count:,} ({pct:.1f}%)")

# Visualize aspect distribution
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Annotation Count', 'Unique Terms'),
    specs=[[{"type": "bar"}, {"type": "bar"}]]
)

# Annotation counts
fig.add_trace(
    go.Bar(x=aspect_counts.index, y=aspect_counts.values,
           marker_color=['#3498db', '#e74c3c', '#2ecc71'],
           text=aspect_counts.values, textposition='outside'),
    row=1, col=1
)

# Unique terms per aspect
unique_terms = train_terms.groupby('aspect_name')['term'].nunique()
fig.add_trace(
    go.Bar(x=unique_terms.index, y=unique_terms.values,
           marker_color=['#3498db', '#e74c3c', '#2ecc71'],
           text=unique_terms.values, textposition='outside'),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False,
                 title_text='GO Term Distribution by Subontology')
fig.update_xaxes(title_text="Subontology")
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Unique Terms", row=1, col=2)
fig.show()

### Part 2: Term Frequency Analysis

**The Long-Tail Problem:**

CAFA exhibits extreme class imbalance - a few terms appear very frequently,
while most terms are rare. This is the classic "long-tail distribution" that
makes this challenge particularly difficult.

**Key Statistics:**
- Most frequent term vs least frequent term ratio
- Percentage of terms appearing < 10 times
- Head vs tail contribution to total annotations

In [ ]:
# Calculate term frequencies
term_freq = train_terms['term'].value_counts()

# Statistics
print("="*70)
print("TERM FREQUENCY STATISTICS")
print("="*70)
print(f"Total unique terms:       {len(term_freq):,}")
print(f"Most frequent term:       {term_freq.iloc[0]:,} proteins")
print(f"Least frequent term:      {term_freq.iloc[-1]:,} proteins")
print(f"Median frequency:         {term_freq.median():.0f} proteins")
print(f"Mean frequency:           {term_freq.mean():.1f} proteins")
print("="*70)

# Rare terms analysis
rare_terms = (term_freq <= 5).sum()
very_rare = (term_freq == 1).sum()
print(f"\nRare Terms Analysis:")
print(f"   Terms appearing ≤5 times:  {rare_terms:,} ({rare_terms/len(term_freq)*100:.1f}%)")
print(f"   Terms appearing once:      {very_rare:,} ({very_rare/len(term_freq)*100:.1f}%)")

# Head vs tail
head_20_pct = term_freq.iloc[:int(len(term_freq)*0.2)].sum()
tail_80_pct = term_freq.iloc[int(len(term_freq)*0.2):].sum()
print(f"\nPareto Analysis:")
print(f"   Top 20% terms cover:       {head_20_pct/term_freq.sum()*100:.1f}% of annotations")
print(f"   Bottom 80% terms cover:    {tail_80_pct/term_freq.sum()*100:.1f}% of annotations")

# Visualization: Log-scale distribution
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(1, len(term_freq) + 1),
    y=term_freq.values,
    mode='lines',
    name='Term Frequency',
    line=dict(color='#3498db', width=2),
    fill='tozeroy'
))

fig.update_layout(
    title='Term Frequency Distribution (Long-Tail)',
    xaxis_title='Term Rank',
    yaxis_title='Frequency (log scale)',
    yaxis_type='log',
    height=500,
    hovermode='x'
)

fig.show()

# Top 20 most frequent terms
print("\nTop 20 Most Frequent Terms:")
top_20 = term_freq.head(20)
for i, (term, count) in enumerate(top_20.items(), 1):
    term_name = go_graph.nodes.get(term, {}).get('name', 'Unknown')
    print(f"   {i:2d}. {term}: {count:5,} | {term_name}")

### Part 3: Information Accretion (IA) Weights

**What is IA?**

Information Accretion measures how "informative" a GO term is:
- **High IA**: Rare, specific terms (hard to predict, high value)
- **Low IA**: Common, general terms (easier to predict, lower value)

The competition metric (F-max) is weighted by IA, meaning:
✅ Correctly predicting rare terms is rewarded more
❌ Missing rare terms is penalized more

**Key Question**: Is there a relationship between term frequency and IA?

In [ ]:
# Merge frequency and IA
term_freq_df = term_freq.reset_index()
term_freq_df.columns = ['term', 'frequency']
merged = term_freq_df.merge(ia_df, on='term', how='left')

# Statistics
print("="*70)
print("INFORMATION ACCRETION (IA) STATISTICS")
print("="*70)
print(f"Terms with IA values:     {merged['IA'].notna().sum():,}")
print(f"Mean IA:                  {ia_df['IA'].mean():.3f}")
print(f"Median IA:                {ia_df['IA'].median():.3f}")
print(f"Max IA:                   {ia_df['IA'].max():.3f}")
print(f"Min IA:                   {ia_df['IA'].min():.3f}")
print("="*70)

# Correlation analysis
corr_pearson = merged[['frequency', 'IA']].corr().iloc[0, 1]
corr_spearman = merged[['frequency', 'IA']].corr(method='spearman').iloc[0, 1]

print(f"\nFrequency vs IA Correlation:")
print(f"   Pearson:  {corr_pearson:.4f}")
print(f"   Spearman: {corr_spearman:.4f}")
print(f"\nNegative correlation confirms: Rare terms have higher IA!")

# Visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Frequency vs IA (log-scale)', 'IA Distribution'),
)

# Scatter plot
sample = merged.sample(min(10000, len(merged)), random_state=42)
fig.add_trace(
    go.Scatter(
        x=sample['frequency'], y=sample['IA'],
        mode='markers',
        marker=dict(size=5, color=sample['IA'], colorscale='Viridis',
                   showscale=True, colorbar=dict(title="IA")),
        text=sample['term'],
        hovertemplate='<b>%{text}</b><br>Freq: %{x}<br>IA: %{y:.3f}<extra></extra>'
    ),
    row=1, col=1
)

# IA distribution
fig.add_trace(
    go.Histogram(x=ia_df['IA'], nbinsx=50, marker_color='lightcoral'),
    row=1, col=2
)

fig.update_xaxes(title_text="Frequency (log)", type="log", row=1, col=1)
fig.update_xaxes(title_text="IA Value", row=1, col=2)
fig.update_yaxes(title_text="IA", row=1, col=1)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_layout(height=500, showlegend=False,
                 title_text='Information Accretion Analysis')
fig.show()

# High-value targets
high_ia = merged.nlargest(15, 'IA')
print("\nTop 15 Highest-Value Terms (High IA):")
for i, row in enumerate(high_ia.itertuples(), 1):
    term_name = go_graph.nodes.get(row.term, {}).get('name', 'Unknown')
    print(f"   {i:2d}. IA={row.IA:.3f} | Freq={row.frequency:3d} | {term_name[:50]}")

### Part 4: Label Cardinality (Multi-Label Nature)

**Multi-Label Classification:**

Unlike traditional classification where each sample has ONE label, in CAFA:
- Each protein can have MULTIPLE GO terms
- Terms can be from different subontologies
- This affects model architecture and evaluation

**Questions to explore:**
- How many labels does a typical protein have?
- Are there proteins with very few or very many labels?
- How does label count vary by subontology?

In [ ]:
# Calculate labels per protein
labels_per_protein = train_terms.groupby('EntryID')['term'].count()

print("="*70)
print("LABEL CARDINALITY STATISTICS")
print("="*70)
print(f"Mean labels per protein:  {labels_per_protein.mean():.2f}")
print(f"Median labels per protein:{labels_per_protein.median():.0f}")
print(f"Min labels:               {labels_per_protein.min()}")
print(f"Max labels:               {labels_per_protein.max()}")
print(f"Std deviation:            {labels_per_protein.std():.2f}")
print("="*70)

# Percentiles
percentiles = labels_per_protein.quantile([0.25, 0.5, 0.75, 0.90, 0.95, 0.99])
print("\nLabel Count Percentiles:")
for p, val in percentiles.items():
    print(f"   {int(p*100)}th percentile: {val:.0f} labels")

# Visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Overall Distribution', 'By Subontology', 
                   'Cumulative Distribution', 'Violin Plot by Aspect'),
    specs=[[{"type": "histogram"}, {"type": "bar"}],
           [{"type": "scatter"}, {"type": "violin"}]]
)

# Overall histogram
fig.add_trace(
    go.Histogram(x=labels_per_protein, nbinsx=50, marker_color='lightblue'),
    row=1, col=1
)

# By subontology
labels_by_aspect = train_terms.groupby(['EntryID', 'aspect_name'])['term'].count().reset_index()
avg_by_aspect = labels_by_aspect.groupby('aspect_name')['term'].mean().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=avg_by_aspect.index, y=avg_by_aspect.values,
           marker_color=['#3498db', '#e74c3c', '#2ecc71'],
           text=[f'{v:.2f}' for v in avg_by_aspect.values],
           textposition='outside'),
    row=1, col=2
)

# Cumulative distribution
sorted_labels = np.sort(labels_per_protein)
cumulative = np.arange(1, len(sorted_labels) + 1) / len(sorted_labels) * 100
fig.add_trace(
    go.Scatter(x=sorted_labels, y=cumulative, mode='lines',
               line=dict(color='green', width=2)),
    row=2, col=1
)

# Violin plot by aspect
for aspect in ['BPO', 'CCO', 'MFO']:
    aspect_data = labels_by_aspect[labels_by_aspect['aspect_name'] == aspect]['term']
    fig.add_trace(
        go.Violin(y=aspect_data, name=aspect, box_visible=True, meanline_visible=True),
        row=2, col=2
    )

fig.update_xaxes(title_text="Number of Labels", row=1, col=1)
fig.update_xaxes(title_text="Subontology", row=1, col=2)
fig.update_xaxes(title_text="Number of Labels", row=2, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Avg Labels", row=1, col=2)
fig.update_yaxes(title_text="Cumulative %", row=2, col=1)
fig.update_yaxes(title_text="Label Count", row=2, col=2)

fig.update_layout(height=800, showlegend=False,
                 title_text='Multi-Label Cardinality Analysis')
fig.show()

print("\nKey Insights:")
print(f"   Most proteins have {labels_per_protein.median():.0f}-{labels_per_protein.quantile(0.75):.0f} labels")
print(f"   {(labels_per_protein == 1).sum():,} proteins have only 1 label")
print(f"   {(labels_per_protein > 20).sum():,} proteins have >20 labels (complex functions)")

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 25px; border-radius: 12px; margin: 25px 0; box-shadow: 0 6px 15px rgba(0,0,0,0.2);">
  <h1 style="color: white; margin: 0; font-size: 32px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
    Gene Ontology Hierarchy
  </h1>
  <p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">
    Exploring the DAG structure and term relationships
  </p>
</div>

Gene Ontology is organized as a **Directed Acyclic Graph (DAG)**:
- **Directed**: Parent → Child relationships (general → specific)
- **Acyclic**: No circular dependencies
- **Multiple inheritance**: A term can have multiple parents

**Key relationships:**
- `is_a`: Child is a type of parent (e.g., "ATP binding" is_a "nucleotide binding")
- `part_of`: Child is part of parent process

**True Path Rule**: If a protein has term X, it also has all ancestors of X

In [ ]:
# Define ontology roots
roots = {
    'BPO': 'GO:0008150',  # biological_process
    'CCO': 'GO:0005575',  # cellular_component
    'MFO': 'GO:0003674'   # molecular_function
}

# Basic graph statistics
print("="*70)
print("GENE ONTOLOGY GRAPH STRUCTURE")
print("="*70)
print(f"Total GO terms (nodes):   {go_graph.number_of_nodes():,}")
print(f"Total relationships:      {go_graph.number_of_edges():,}")
print(f"Avg edges per node:       {go_graph.number_of_edges()/go_graph.number_of_nodes():.2f}")
print("="*70)

# Analyze each subontology
print("\nSubontology Statistics:")
for name, root in roots.items():
    if root in go_graph:
        # Get all descendants
        descendants = nx.descendants(go_graph, root)
        descendants.add(root)
        
        # Count terms in training set
        aspect_code = {'BPO': 'P', 'CCO': 'C', 'MFO': 'F'}[name]
        terms_in_train = train_terms[train_terms['aspect'] == aspect_code]['term'].nunique()
        
        print(f"\n   {name}:")
        print(f"      Total terms in GO:     {len(descendants):,}")
        print(f"      Used in training:      {terms_in_train:,} ({terms_in_train/len(descendants)*100:.1f}%)")

# Sample term exploration
sample_term = 'GO:0005515'  # protein binding
if sample_term in go_graph:
    term_info = go_graph.nodes[sample_term]
    print(f"\nExample Term: {sample_term}")
    print(f"   Name: {term_info.get('name', 'N/A')}")
    print(f"   Namespace: {term_info.get('namespace', 'N/A')}")
    print(f"   Definition: {term_info.get('def', 'N/A')[:100]}...")
    
    # Parents and children
    parents = list(go_graph.predecessors(sample_term))
    children = list(go_graph.successors(sample_term))
    print(f"   Parent terms: {len(parents)}")
    print(f"   Child terms: {len(children)}")

<div style="background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%); padding: 25px; border-radius: 12px; margin: 25px 0; box-shadow: 0 6px 15px rgba(0,0,0,0.2);">
  <h1 style="color: #334155; margin: 0; font-size: 32px; text-shadow: 1px 1px 2px rgba(255,255,255,0.5);">
    Term Depth Analysis
  </h1>
  <p style="color: #475569; margin: 10px 0 0 0; font-size: 16px;">
    Understanding hierarchy depth and IA relationships
  </p>
</div>

**Depth** = shortest path from root to term

**Why depth matters:**
- Shallow terms (depth 1-3): General, common functions
- Medium depth (4-7): Specific functional categories  
- Deep terms (8+): Highly specific, rare functions

**Hypothesis**: Deeper terms should have higher IA (more informative)

In [ ]:
def calculate_depths(graph, root):
    """Calculate shortest path from root to all descendants"""
    depths = {}
    if root not in graph:
        return depths
    
    depths[root] = 0
    
    # Get all nodes that can reach the root
    for node in graph.nodes():
        if node == root:
            continue
        try:
            # In obonet, edges point FROM child TO parent
            depth = nx.shortest_path_length(graph, node, root)
            depths[node] = depth
        except nx.NetworkXNoPath:
            pass
    
    return depths

# Calculate depths for all subontologies
all_depths = {}
for name, root in roots.items():
    depths = calculate_depths(go_graph, root)
    all_depths[name] = depths
    
    if depths:
        print(f"\n{name} Depth Statistics:")
        depth_values = list(depths.values())
        print(f"   Max depth:     {max(depth_values)}")
        print(f"   Mean depth:    {np.mean(depth_values):.2f}")
        print(f"   Median depth:  {np.median(depth_values):.0f}")

# Create depth dataframe for training terms
depth_data = []
for _, row in train_terms.iterrows():
    term = row['term']
    aspect = row['aspect_name']
    
    if aspect in all_depths and term in all_depths[aspect]:
        depth = all_depths[aspect][term]
        depth_data.append({
            'term': term,
            'aspect': aspect,
            'depth': depth
        })

depth_df = pd.DataFrame(depth_data)

# Check if we have data
if len(depth_df) == 0:
    print("\nWarning: No depth data calculated. Check GO graph structure.")
else:
    print(f"\nCalculated depths for {len(depth_df):,} terms")

# Merge with IA (only if we have depth data)
if len(depth_df) > 0:
    depth_ia = depth_df.merge(ia_df, on='term', how='left')
    
    # Visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Depth Distribution by Subontology', 'Depth vs IA Relationship')
    )
    
    # Depth distribution
    for aspect in ['BPO', 'CCO', 'MFO']:
        aspect_depths = depth_df[depth_df['aspect'] == aspect]['depth']
        if len(aspect_depths) > 0:
            fig.add_trace(
                go.Violin(y=aspect_depths, name=aspect, box_visible=True, meanline_visible=True),
                row=1, col=1
            )
    
    # Depth vs IA scatter
    depth_ia_clean = depth_ia.dropna()
    if len(depth_ia_clean) > 0:
        sample = depth_ia_clean.sample(min(5000, len(depth_ia_clean)), random_state=42)
        fig.add_trace(
            go.Scatter(
                x=sample['depth'], y=sample['IA'],
                mode='markers',
                marker=dict(size=4, color=sample['IA'], colorscale='Viridis',
                           opacity=0.6),
                showlegend=False
            ),
            row=1, col=2
        )
        
        # Add trend line
        from scipy import stats
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            depth_ia_clean['depth'], depth_ia_clean['IA']
        )
        line_x = np.array([depth_ia_clean['depth'].min(), depth_ia_clean['depth'].max()])
        line_y = slope * line_x + intercept
        fig.add_trace(
            go.Scatter(x=line_x, y=line_y, mode='lines',
                       line=dict(color='red', width=2, dash='dash'),
                       name=f'Trend (R²={r_value**2:.3f})'),
            row=1, col=2
        )
        
        print(f"\nDepth-IA Correlation: {r_value:.3f}")
        print(f"   Positive correlation confirms: Deeper terms have higher IA!")
    
    fig.update_xaxes(title_text="Subontology", row=1, col=1)
    fig.update_xaxes(title_text="Depth", row=1, col=2)
    fig.update_yaxes(title_text="Depth", row=1, col=1)
    fig.update_yaxes(title_text="IA", row=1, col=2)
    fig.update_layout(height=500, title_text='GO Term Depth Analysis')
    fig.show()
else:
    print("Skipping visualization due to missing depth data")

<div style="background: linear-gradient(135deg, #43e97b 0%, #38f9d7 100%); padding: 25px; border-radius: 12px; margin: 25px 0; box-shadow: 0 6px 15px rgba(0,0,0,0.2);">
  <h1 style="color: white; margin: 0; font-size: 32px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
    Taxonomy Analysis
  </h1>
  <p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">
    Understanding species distribution in training data
  </p>
</div>


**Why taxonomy matters:**
- Proteins from related species often have similar functions
- Evolutionary conservation can be leveraged for prediction
- Understanding species distribution helps assess model generalization

**Model organisms** (well-studied species) dominate the training data

In [ ]:
# Species frequency
species_counts = train_tax['TaxonID'].value_counts()

print("="*70)
print("TAXONOMY STATISTICS")
print("="*70)
print(f"Total proteins:           {len(train_tax):,}")
print(f"Unique species (taxa):    {len(species_counts):,}")
print(f"Most common taxon:        {species_counts.iloc[0]:,} proteins")
print(f"Median proteins per taxon:{species_counts.median():.0f}")
print("="*70)

# Top species (common model organisms)
print("\nTop 15 Species by Protein Count:")
top_species = species_counts.head(15)

# Common model organism IDs
model_organisms = {
    9606: 'Homo sapiens (Human)',
    10090: 'Mus musculus (Mouse)',
    10116: 'Rattus norvegicus (Rat)',
    7227: 'Drosophila melanogaster (Fruit fly)',
    6239: 'Caenorhabditis elegans (Nematode)',
    3702: 'Arabidopsis thaliana (Plant)',
    559292: 'Saccharomyces cerevisiae (Yeast)',
    83333: 'Escherichia coli K-12',
    284812: 'Schizosaccharomyces pombe (Fission yeast)'
}

for i, (taxon, count) in enumerate(top_species.items(), 1):
    name = model_organisms.get(taxon, f'TaxonID {taxon}')
    pct = count / len(train_tax) * 100
    print(f"   {i:2d}. {name:40s}: {count:6,} ({pct:5.2f}%)")

# Visualization
fig = go.Figure()

fig.add_trace(go.Bar(
    y=[model_organisms.get(t, f'Taxon {t}') for t in top_species.index],
    x=top_species.values,
    orientation='h',
    marker_color='lightseagreen',
    text=top_species.values,
    textposition='outside'
))

fig.update_layout(
    title='Top 15 Species in Training Data',
    xaxis_title='Number of Proteins',
    yaxis_title='Species',
    height=600,
    yaxis={'categoryorder':'total ascending'}
)

fig.show()

# Distribution analysis
print("\nDistribution Analysis:")
top_10_pct = species_counts.head(10).sum() / len(train_tax) * 100
top_50_pct = species_counts.head(50).sum() / len(train_tax) * 100
print(f"   Top 10 species cover:     {top_10_pct:.1f}% of proteins")
print(f"   Top 50 species cover:     {top_50_pct:.1f}% of proteins")
print(f"   Species with 1 protein:   {(species_counts == 1).sum():,}")
print(f"   Species with <10 proteins:{(species_counts < 10).sum():,}")

<div style="background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); padding: 25px; border-radius: 12px; margin: 25px 0; box-shadow: 0 6px 15px rgba(0,0,0,0.2);">
  <h1 style="color: white; margin: 0; font-size: 32px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
    Term Co-occurrence Analysis
  </h1>
  <p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">
    Discovering functional relationships between GO terms
  </p>
</div>

**Co-occurrence patterns reveal:**
- Terms that commonly appear together
- Potential functional modules
- Cross-ontology relationships

This information can help:
- Feature engineering (term combinations)
- Model architecture (multi-task learning)
- Post-processing (co-prediction rules)

In [ ]:
# Sample proteins for co-occurrence analysis
sample_size = min(10000, train_terms['EntryID'].nunique())
sample_proteins = train_terms['EntryID'].unique()[:sample_size]
sample_data = train_terms[train_terms['EntryID'].isin(sample_proteins)]

# Calculate co-occurrence
print("Calculating term co-occurrence patterns...")
co_occurrence = Counter()

for protein, group in tqdm(sample_data.groupby('EntryID'), desc="Processing"):
    terms = group['term'].tolist()
    if len(terms) > 1:
        # Count all pairs
        for i in range(len(terms)):
            for j in range(i+1, len(terms)):
                pair = tuple(sorted([terms[i], terms[j]]))
                co_occurrence[pair] += 1

# Top co-occurring pairs
top_pairs = sorted(co_occurrence.items(), key=lambda x: x[1], reverse=True)[:20]

print("\nTop 20 Co-occurring Term Pairs:")
print(f"{'Rank':<6}{'Count':<8}{'Term 1':<15}{'Term 2':<15}{'Names'}")
print("="*100)

for i, (pair, count) in enumerate(top_pairs, 1):
    term1, term2 = pair
    name1 = go_graph.nodes.get(term1, {}).get('name', 'Unknown')[:20]
    name2 = go_graph.nodes.get(term2, {}).get('name', 'Unknown')[:20]
    print(f"{i:<6}{count:<8}{term1:<15}{term2:<15}{name1} + {name2}")

# Cross-ontology co-occurrence
print("\nCross-Ontology Co-occurrence:")
cross_onto = defaultdict(int)

for protein, group in sample_data.groupby('EntryID'):
    aspects = set(group['aspect'].tolist())
    if len(aspects) > 1:
        for asp in aspects:
            cross_onto[asp] += 1

total_proteins = len(sample_data['EntryID'].unique())
print(f"   Proteins with MF+BP+CC: {len(sample_data.groupby('EntryID')['aspect'].apply(lambda x: len(set(x)) == 3))}")
print(f"   Proteins with 2 aspects: {len(sample_data.groupby('EntryID')['aspect'].apply(lambda x: len(set(x)) == 2))}")
print(f"   Proteins with 1 aspect:  {len(sample_data.groupby('EntryID')['aspect'].apply(lambda x: len(set(x)) == 1))}")

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 25px; border-radius: 12px; margin: 25px 0; box-shadow: 0 6px 15px rgba(0,0,0,0.2);">
  <h1 style="color: white; margin: 0; font-size: 32px; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
    Train-Test Analysis
  </h1>
  <p style="color: rgba(255,255,255,0.9); margin: 10px 0 0 0; font-size: 16px;">
    Comparing distributions and identifying potential shifts
  </p>
</div>

**Important**: This is a **prospective evaluation**
- Test proteins currently have NO annotations
- During evaluation, only proteins that gain experimental validation will be scored
- We can still analyze distributional properties

**Key questions:**
- How do test sequences compare to training sequences?
- Is there protein ID overlap?
- Are sequence length distributions similar?

In [ ]:
# Load test sequences
print("Loading test sequences...")
test_seqs = []
for record in tqdm(SeqIO.parse(paths['test_seq'], 'fasta'), desc="Loading test set"):
    entry_id = record.id.split('|')[1] if '|' in record.id else record.id
    test_seqs.append({
        'EntryID': entry_id,
        'length': len(record.seq)
    })

test_df = pd.DataFrame(test_seqs)

# Basic statistics
print("\n" + "="*70)
print("TRAIN-TEST COMPARISON")
print("="*70)
print(f"Training proteins:        {len(train_seqs):,}")
print(f"Test proteins:            {len(test_df):,}")
print(f"Test/Train ratio:         {len(test_df)/len(train_seqs):.2f}x")
print("="*70)

# Protein ID overlap
train_ids = set(train_seqs['EntryID'])
test_ids = set(test_df['EntryID'])
overlap = train_ids.intersection(test_ids)

print(f"\nProtein ID Overlap:")
print(f"   Overlapping proteins:     {len(overlap):,}")
print(f"   % of train in test:       {len(overlap)/len(train_ids)*100:.2f}%")
print(f"   % of test in train:       {len(overlap)/len(test_ids)*100:.2f}%")
print(f"   New proteins in test:     {len(test_ids - train_ids):,}")

# Sequence length comparison
print(f"\nSequence Length Comparison:")
print(f"{'Metric':<20}{'Train':<15}{'Test':<15}{'Difference'}")
print("="*70)
print(f"{'Mean':<20}{train_seqs['length'].mean():<15.1f}{test_df['length'].mean():<15.1f}{test_df['length'].mean() - train_seqs['length'].mean():.1f}")
print(f"{'Median':<20}{train_seqs['length'].median():<15.0f}{test_df['length'].median():<15.0f}{test_df['length'].median() - train_seqs['length'].median():.0f}")
print(f"{'Std Dev':<20}{train_seqs['length'].std():<15.1f}{test_df['length'].std():<15.1f}{test_df['length'].std() - train_seqs['length'].std():.1f}")
print(f"{'Min':<20}{train_seqs['length'].min():<15}{test_df['length'].min():<15}{test_df['length'].min() - train_seqs['length'].min()}")
print(f"{'Max':<20}{train_seqs['length'].max():<15,}{test_df['length'].max():<15,}{test_df['length'].max() - train_seqs['length'].max():,}")

# Visualization
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=train_seqs['length'],
    name='Train',
    opacity=0.7,
    marker_color='lightblue',
    nbinsx=100
))

fig.add_trace(go.Histogram(
    x=test_df['length'],
    name='Test',
    opacity=0.7,
    marker_color='lightcoral',
    nbinsx=100
))

fig.update_layout(
    title='Sequence Length Distribution: Train vs Test',
    xaxis_title='Sequence Length',
    yaxis_title='Count',
    barmode='overlay',
    height=500
)

fig.show()

print("\nGood news: Distributions are very similar!")
print("   No severe distribution shift")
print("   Model should generalize well")